# YOLO HAND SIGN PREDICTION

## Install Necessary Packages 

In [1]:
pip install opencv-python==4.6.0.66

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install PyYAML

## Import Necessary Libraries

In [3]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

## STEP 1 : LOAD YAML FILE

In [4]:
with open('data.yaml', mode = 'r') as f:
    data_yaml = yaml.load(f, Loader = SafeLoader)
    
labels = data_yaml['names']
print(labels)

['Hello', 'IloveYou', 'No', 'Please', 'Thanks', 'Yes']


## STEP2 : LOAD YOLO MODEL with OPENCV

In [5]:
yolo = cv2.dnn.readNetFromONNX('./Model3/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

## STEP 3: Load the image

In [6]:
img = cv2.imread('./Hand Sign Test.jpg')
image = img.copy()

In [7]:
# To View the image

'''
cv2.imshow('image',image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\ncv2.imshow('image',image)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

## STEP 4 : Retune the Image

In [8]:
# Check Height Width and Depth of image

row, col, d = image.shape
row, col, d

(351, 441, 3)

### TASK : Covert our image into Square Image

In [9]:
# 1. decide the larger between row, col
max_rc = max(row, col)

# 2. Black Shell Matrix
input_image = np.zeros((max_rc, max_rc, 3), dtype = np.uint8)

In [10]:
input_image [0:row, 0:col] = image

In [11]:

cv2.imshow('input_image',input_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [12]:

INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image , 1/255, (INPUT_WH_YOLO, INPUT_WH_YOLO), swapRB=True, crop= False)
yolo.setInput(blob)

# Prediction from YOLO
preds = yolo.forward() 

In [13]:
print(preds)

[[[4.2777500e+00 6.1805229e+00 2.3314631e+01 ... 1.0987000e-01
   7.7123858e-02 1.8501990e-01]
  [1.3012996e+01 6.2164574e+00 2.6709650e+01 ... 1.2223515e-01
   9.9219620e-02 1.2808347e-01]
  [2.0362461e+01 4.5874434e+00 1.4012946e+01 ... 6.3468076e-02
   7.8280710e-02 8.7253481e-02]
  ...
  [5.5472833e+02 6.1737140e+02 2.9396722e+02 ... 1.1254704e-01
   1.3156779e-01 1.1397129e-01]
  [5.8562805e+02 6.1966803e+02 2.3207990e+02 ... 9.8370858e-02
   1.3282637e-01 1.1745501e-01]
  [6.1659662e+02 6.1853528e+02 3.0048816e+02 ... 7.3034920e-02
   1.3365467e-01 1.0952050e-01]]]


In [14]:
# Conclusion : 

'''
No of Bounding Boxes Detected by Model = 25200

for each and every Bounding Box those info are available in 25 coloumns

'''

'\nNo of Bounding Boxes Detected by Model = 25200\n\nfor each and every Bounding Box those info are available in 25 coloumns\n\n'

<img src = 'Supporting Image\Yolo Structure.png'>

## STEP 5:  NON MAX SUPPRESSION 

### Filter Detection based on Confidence 0.4 and Probability Score 0.25

In [15]:
detections = preds[0]

boxes = []
confidences = []
classes = []

In [16]:
# Caclculate Width and Height of the Image

image_w, image_h = input_image.shape[:2]
image_w, image_h

(441, 441)

In [17]:
# Calculate X_Factor & Y_Factor

x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

x_factor, y_factor

(0.6890625, 0.6890625)

In [18]:
# Filter Detection based on Confidence 0.4 and Probability Score 0.25

for i in range(len(detections)):
    row = detections[i]
    
    # Confidence of detecting an Object
    confidence = row[4] 
    
    if confidence > 0.4: 
        
        # Maximum Probability from 20 Objects
        class_score = row[5:].max()
        
        # Which Class gives Max Probablitiy
        class_id = row[5:].argmax()
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            
            # CONSTRUCT BOUNDING BOXES from the values
            # Left, TOP, WIDTH & HEIGHT
            
            left   = int((cx - 0.5*w)* x_factor)
            top    = int((cy - 0.5*h)* y_factor)
            width  = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left, top, width, height])
            
            # Append Values into List
            confidences.append (confidence)
            boxes.append (box)
            classes.append (class_id)
       

### Clean Duplicate Values & Store it in Numpy List 

In [19]:
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

### Non Max Suppression Operation

In [20]:
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.45).flatten()

In [21]:
len(index)

3

### Draw the BOUNDING BOX

In [22]:
for ind in index : 
    
    # Extract Bounding Box
    x, y, w, h = boxes_np[ind]
    bb_conf = confidences_np[ind] * 100
    classes_id = classes[ind]
    class_name = labels[classes_id]
    
    text = f'{class_name} : {bb_conf}%'
    
    cv2.rectangle(image, (x,y), (x+w, y+h), (0,255, 0), 2)
    cv2.rectangle(image, (x,y-30), (x+w, y), (255,255, 255), -1)
    
    cv2.putText(image, text, (x,y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0,0,0),1)

In [23]:
cv2.imshow('Original',img)
cv2.imshow('Predicted',image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## The END